## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-20-43-33 +0000,nypost,Pokémon thief catches them all as he swipes $1...,https://nypost.com/2025/07/14/business/pokemon...
1,2025-07-14-20-41-59 +0000,nyt,It’s No Bluff: The Tariff Rate Is Soaring Unde...,https://www.nytimes.com/2025/07/14/us/politics...
2,2025-07-14-20-37-28 +0000,nyt,The Economy Has Been Resilient. The New Round ...,https://www.nytimes.com/2025/07/14/us/politics...
3,2025-07-14-20-35-00 +0000,bbc,Gaza father's outrage after Israeli strike kil...,https://www.bbc.com/news/articles/ckglpk9xjewo
4,2025-07-14-20-31-06 +0000,nyt,"Fauja Singh, Master Runner at an Advanced Age,...",https://www.nytimes.com/2025/07/14/sports/fauj...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2377/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,14
73,elon,5
74,musk,5
62,ceo,4
172,fire,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
58,2025-07-14-02-11-23 +0000,nypost,Trump economic adviser Kevin Hassett says prez...,https://nypost.com/2025/07/13/business/trump-e...,36
41,2025-07-14-13-49-32 +0000,nypost,YouTube minting billions off pirated videos pl...,https://nypost.com/2025/07/14/business/youtube...,33
26,2025-07-14-16-44-53 +0000,nypost,Jerome Powell asks inspector general to review...,https://nypost.com/2025/07/14/business/fed-cha...,32
40,2025-07-14-14-05-01 +0000,nypost,Inside Linda Yaccarino’s rocky two-year stint ...,https://nypost.com/2025/07/14/business/inside-...,32
17,2025-07-14-18-04-03 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,29


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
58,36,2025-07-14-02-11-23 +0000,nypost,Trump economic adviser Kevin Hassett says prez...,https://nypost.com/2025/07/13/business/trump-e...
40,32,2025-07-14-14-05-01 +0000,nypost,Inside Linda Yaccarino’s rocky two-year stint ...,https://nypost.com/2025/07/14/business/inside-...
0,19,2025-07-14-20-43-33 +0000,nypost,Pokémon thief catches them all as he swipes $1...,https://nypost.com/2025/07/14/business/pokemon...
41,19,2025-07-14-13-49-32 +0000,nypost,YouTube minting billions off pirated videos pl...,https://nypost.com/2025/07/14/business/youtube...
2,15,2025-07-14-20-37-28 +0000,nyt,The Economy Has Been Resilient. The New Round ...,https://www.nytimes.com/2025/07/14/us/politics...
44,15,2025-07-14-13-10-28 +0000,nypost,Popular supermarket chain adding AI ‘smart car...,https://nypost.com/2025/07/14/lifestyle/wegman...
29,13,2025-07-14-16-08-41 +0000,bbc,Dozens killed in clashes between Druze and Bed...,https://www.bbc.com/news/articles/cgmwp70pwmyo
54,13,2025-07-14-08-01-26 +0000,bbc,Don't vilify Air India crash crew: Indian pilo...,https://www.bbc.com/news/articles/cddzp0g5l25o
3,12,2025-07-14-20-35-00 +0000,bbc,Gaza father's outrage after Israeli strike kil...,https://www.bbc.com/news/articles/ckglpk9xjewo
6,12,2025-07-14-20-25-16 +0000,nypost,Starbucks orders four-day office return for em...,https://nypost.com/2025/07/14/business/starbuc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
